In [9]:
import numpy as np
import pandas as pd
pd.options.display.float_format = "{:.4f}".format
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,10)
plt.style.use("~/.dracula.mplstyle")
import statsmodels.api as sm
import scipy.stats as scp
import seaborn as sns

In [10]:
class DataImport:
    
    def __init__(self, filename, add_ind=[], na_method="interpolate"):
        self.xlsx = pd.ExcelFile(filename)
        self.add_ind = add_ind # Additional indices to try to 
        
        print(f"Read in <{filename}>.")
        self.sheets = []
        print('-'*20, "Reading Sheets ", '-'*20)
        self.read_sheets()
        print('-'*20, "Setting Indices", '-'*20)
        self.set_indices()
        print('-'*20, "Checking Nulls ", '-'*20)
        self.check_nulls(na_method)
        print("\n\n")
    
    def read_sheets(self):
        for i, s in enumerate(self.xlsx.sheet_names):
            read_sheet = self.xlsx.parse(s)
            print(f"Sheet {i}: {s}; shape={read_sheet.shape}")
            self.sheets.append(read_sheet)
    
    def set_indices(self):
        for s in self.sheets:
            for d in ["date", "Date", "Unnamed: 0"]+self.add_ind:
                try:
                    s.set_index(d, inplace=True) # Date as index
                    break
                except KeyError:
                    continue
    
    def check_nulls(self, method):
        n = 0
        for i, s in enumerate(self.sheets[1:]):
            has_nulls = True
            while has_nulls:
                if sum(s.isna().sum())==0: # Check for nulls
                    has_nulls = False
                else:
                    if method=="interpolate":
                        print(f"Null values found in <{i+1}>. Interpolating linearly.")
                        s = s.interpolate() # Linerly interpolate nulls
                        self.sheets[i+1] = s
                    elif method=="drop":
                        print(f"Null values found in <{i+1}>. Dropping.")
                        s = s.dropna()
                        self.sheets[i+1] = s
                    elif method=="none":
                        break
                    n += 1
                if n>i+1:
                    print("Something broke...")
                    n=i+1
                    break
        if method=="none": print("Did not check for nulls.")
        elif n==0: print("No nulls found :)")

xl = DataImport("gmo.xlsx", na_method="none")
display(xl.sheets[0])
sig = xl.sheets[1]
ret = xl.sheets[2]
ret = ret.subtract(xl.sheets[3]["US3M"], axis=0)

Read in <gmo.xlsx>.
-------------------- Reading Sheets  --------------------
Sheet 0: descriptions; shape=(6, 4)
Sheet 1: signals; shape=(345, 4)
Sheet 2: returns (total); shape=(345, 3)
Sheet 3: risk-free rate; shape=(345, 2)
-------------------- Setting Indices --------------------
-------------------- Checking Nulls  --------------------
Did not check for nulls.





,Unit,Type,Description
Unnamed: 0,,,
DP,Ratio,Index,Dividend-Price Ratio of the S&P500
EP,Ratio,Index,Earnings-Price Ratio of the S&P500
US10Y,Yield,Index,10-Year Tnotes
SPY,Total Return,ETF,S&P 500
GMWAX,Total Return,Mutual Fund,GMO
RF,Total Return,Index,3-Month Tbills


In [11]:
def pivot_summaries(data, year_mask=None, pi=0.05, nper=12):
    if not(year_mask):
        year_mask = [(str(min(data.index).year), str(max(data.index).year))]
    
    data_melt = pd.melt(data, ignore_index=False).sort_index()
    
    pivlist = []
    for start, end in year_mask:
        def me(x): return nper * np.mean(x)
        def sd(x): return np.sqrt(nper) * np.std(x, ddof=1) # account for degrees of freedom
        def sr(x): return np.sqrt(nper) * np.mean(x) / np.std(x, ddof=1)
        def sk(x): return scp.skew(x, bias=False)

        af = [me, sd, sr, sk]
        af_names = ["Mean", "Vol", "Sharpe", "Skew"]
        
        piv = pd.pivot_table(data_melt[start:end], index="variable", values="value",
                             aggfunc=af)
        piv.columns = pd.MultiIndex.from_arrays([[f"{start}-{end}"] * len(af), af_names],
                                                names=["year", "measure"])
        pivlist.append(piv)

    summ = pd.concat(pivlist, axis=1)
    return summ

def year_to_index(df):
    return df.T.reset_index(level="year").pivot(columns="year").T

miny = str(sig.index.min().year)
maxy = str(sig.index.max().year)

year_mask = [(miny, "2011"), ("2012", maxy), (miny, maxy)]
ps = pivot_summaries(ret, year_mask)
ps = year_to_index(ps)
display(ps)

measure              Mean  Sharpe    Skew    Vol
variable year                                   
GMWAX    1993-2011 0.0158  0.1266     NaN 0.1250
         1993-2021 0.0329  0.2964     NaN 0.1111
         2012-2021 0.0593  0.6952 -0.9420 0.0853
SPY      1993-2011 0.0538  0.3528 -0.5863 0.1526
         1993-2021 0.0883  0.6044 -0.6025 0.1461
         2012-2021 0.1545  1.1785 -0.5192 0.1311

In [12]:
def tailrisk(v, prob, logret=False, exkurt=True):
    """Arguments:
    v: pd.DataFrame containing columns of returns
    prob: float percentile for VaR
    excess: bool whether kurtosis of normal = 0 (True) or = 3 (False)
    """
    try:
        c = v.columns
    except AttributeError:
        c = ["Portfolio"]
    
    mi = v.min()
#     sk = v.skew()
#     ku = v.kurtosis() - 3 * exkurt
    var = v.quantile(prob)
    
    cvar = []
    mask =  v < v.quantile(prob)
    for name, col in mask.iteritems():
        below = v[name].loc[col]
        cvar.append(sum(below)/len(below))
    
    cvar = pd.Series(cvar, index=c)
    
    if logret:
        cumu = np.exp(v.cumsum())
    else:
        cumu = (v+1).cumprod()
    from_peak = (cumu - cumu.cummax()) / cumu.cummax()

    pl, rl, dl = [], [], []
    trough = from_peak.idxmin()
    for col, date in trough.iteritems():
        peak = max(v.loc[(from_peak.index < date) & (from_peak[col] == 0), col].index)
        try:
            reco = min(v.loc[(from_peak.index > date) & (from_peak[col] == 0), col].index)
        except ValueError:
            reco = None
        
        if logret:
            # This may not work right at the moment...
            draw = np.log(cumu.loc[date,col] / cumu.loc[peak,col])
        else:
            draw = (cumu.loc[date,col] - cumu.loc[peak,col]) / cumu.loc[peak,col]

        pl.append(peak); rl.append(reco); dl.append(draw)

    dl = pd.Series(dl, index=c)
    pl = pd.Series(pl, index=c)
    rl = pd.Series(rl, index=c)
    
    ret = pd.DataFrame([mi, var, cvar, dl],
                       index=["min", f"var_{prob}", f"cvar_{prob}", "drawdown"],
                       columns=c)
    dra = pd.DataFrame([pl, trough, rl],
                       index=["peak", "trough", "recovery"],
                       columns=c)
    ret = pd.concat([ret], axis=0).T
    
    return ret

tr = pd.DataFrame()
for y in year_mask:
    df = tailrisk(ret[y[0]:y[1]], 0.05, logret=False)
    df.columns = pd.MultiIndex.from_arrays([[f"{y[0]}-{y[1]}"] * len(df.columns), df.columns],
                                                names=["year", "measure"])
    tr = pd.concat([tr, df], axis=1)
tr

year    1993-2011                             2012-2021                     \
measure       min var_0.05 cvar_0.05 drawdown       min var_0.05 cvar_0.05   
SPY       -0.1656  -0.0783   -0.1008  -0.5600   -0.1247  -0.0602   -0.0821   
GMWAX     -0.1492  -0.0598   -0.0965  -0.4729   -0.1187  -0.0306   -0.0540   

year             1993-2021                              
measure drawdown       min var_0.05 cvar_0.05 drawdown  
SPY      -0.1963   -0.1656  -0.0696   -0.0956  -0.5600  
GMWAX    -0.1696   -0.1492  -0.0449   -0.0858  -0.4729

In [27]:
def portstats_bm(v, bm, nper=12):
    """Arguments:
    v: pd.DataFrame containing rows of returns OR
    bm: pd.DataFrame containing rows of returns
    nper: int for annualization
    """
    try:
        c = v.columns
    except AttributeError:
        c = ["Portfolio"]
    
    mu = v.mean() * nper
    sig = v.std() * np.sqrt(nper)
    
    alpha, rsq, mae, tr, ir, reg = [], [], [], [], [], []
    beta = {}
    for col in bm.columns:
        beta[col] = []
    
    for name, col in v.iteritems():
        res = sm.OLS(col, sm.add_constant(bm.values), missing="drop").fit()
        
        alpha.append(res.params[0] * nper)
        rsq.append(res.rsquared)
        mae.append(abs(res.params[0] * nper))
        for i, col in enumerate(bm.columns):
            beta[col].append(res.params[i+1])
        tr.append(mu[name] / res.params[1])
        ir.append(res.params[0] / res.resid.std() * np.sqrt(nper))
        reg.append(res)
    
    alpha = pd.Series(alpha, index=c)
    beta = pd.DataFrame(beta, index=c).T
    beta.index = ["beta_"+i for i in beta.index]
    mae = pd.Series(mae, index=c)
    tr = pd.Series(tr, index=c)
    ir = pd.Series(ir, index=c)
    
    ret = pd.DataFrame([mu, sig, mu / sig, alpha],
                       index=["mean", "std", "sharpe", "alpha"],
                       columns=c)
    end = pd.DataFrame([rsq, mae, tr, ir], index=["r2", "mae", "treynor", "info ratio"], columns=c)
    
    ret = pd.concat([ret, beta, end]).T
    return ret, reg

re = pd.DataFrame()
for y in year_mask:
    tdf = ret[y[0]:y[1]]
    rdf, reg = portstats_bm(tdf[["GMWAX"]], tdf[["SPY"]])
    rdf = rdf[["alpha", "beta_SPY", "r2"]]
    rdf.columns = pd.MultiIndex.from_arrays([[f"{y[0]}-{y[1]}"] * len(rdf.columns), rdf.columns],
                                                names=["year", "measure"])
    re = pd.concat([re, rdf], axis=1)
re

year    1993-2011                 2012-2021                 1993-2021  \
measure     alpha beta_SPY     r2     alpha beta_SPY     r2     alpha   
GMWAX     -0.0058   0.5396 0.5071   -0.0285   0.5683 0.7633   -0.0135   

year                     
measure beta_SPY     r2  
GMWAX     0.5461 0.5668

In [ ]:
def portstats_bm(v, bm, nper=12, logret=False):
    """Arguments:
    v: pd.DataFrame containing rows of returns OR
    bm: pd.DataFrame containing rows of returns
    nper: int for annualization
    """
    try:
        c = v.columns
    except AttributeError:
        c = ["Portfolio"]
    
    mu = v.mean() * nper
    sig = v.std() * np.sqrt(nper)
    
    alpha, rsq, mae, tr, ir, reg = [], [], [], [], [], []
    beta = {}
    for col in bm.columns:
        beta[col] = []
    
    for name, col in v.iteritems():
        res = sm.OLS(col, sm.add_constant(bm.values), missing="drop").fit()
        
        alpha.append(res.params[0] * nper)
        rsq.append(res.rsquared)
        mae.append(abs(res.params[0] * nper))
        for i, col in enumerate(bm.columns):
            beta[col].append(res.params[i+1])
        tr.append(mu[name] / res.params[1])
        ir.append(res.params[0] / res.resid.std() * np.sqrt(nper))
        reg.append(res)
    
    alpha = pd.Series(alpha, index=c)
    beta = pd.DataFrame(beta, index=c).T
    beta.index = ["beta_"+i for i in beta.index]
    mae = pd.Series(mae, index=c)
    tr = pd.Series(tr, index=c)
    ir = pd.Series(ir, index=c)
    
    if logret:
        cumu = np.exp(v.cumsum())
    else:
        cumu = (v+1).cumprod()
    from_peak = (cumu - cumu.cummax()) / cumu.cummax()

    pl, rl, dl = [], [], []
    trough = from_peak.idxmin()
    for col, date in trough.iteritems():
        pk = v.loc[(from_peak.index < date) & (from_peak[col] == 0), col].index
        if len(pk):
            peak = max(pk)
        else:
            peak = max(v.index)
            date = peak
        
        try:
            reco = min(v.loc[(from_peak.index > date) & (from_peak[col] == 0), col].index)
        except ValueError:
            reco = None
        
        if logret:
            # This may not work right at the moment...
            draw = np.log(cumu.loc[date,col] / cumu.loc[peak,col])
        else:
            draw = (cumu.loc[date,col] - cumu.loc[peak,col]) / cumu.loc[peak,col]

        pl.append(peak); rl.append(reco); dl.append(draw)

    dl = pd.Series(dl, index=c)
    pl = pd.Series(pl, index=c)
    rl = pd.Series(rl, index=c)
    
    ret = pd.DataFrame([mu, sig, mu / sig, alpha],
                       index=["mean", "std", "sharpe", "alpha"],
                       columns=c)
    end = pd.DataFrame([rsq, mae, tr, ir, dl], index=["r2", "mae", "treynor", "info ratio", "drawdown"], columns=c)
    dra = pd.DataFrame([pl, trough, rl],
                       index=["peak", "trough", "recovery"],
                       columns=c)
    ret = pd.concat([ret, beta, end, dra]).T
    return ret, reg